# DataGrid

In [ ]:
from ipywxyz import DataGrid, AlphaColorPicker
import pandas as pd, traitlets as T, ipywidgets as W

In [ ]:
dg = DataGrid(layout=dict(height="400px", flex="1"))

with W.Output():
    @W.interact
    def data(rows=(1, 1000), cols=(1, 1000)):
        dg.value = pd.DataFrame(pd.np.arange(rows * cols).reshape((rows, cols)))
x = W.IntSlider(description='scroll x')
y = W.IntSlider(description='scroll y', orientation='vertical')
rs = W.IntSlider(description='row height', orientation='vertical')
cs = W.IntSlider(description='column width')
hr = W.HTML()
hc = W.HTML()

T.link((x, "value"), (dg, "scroll_x"))
T.dlink((dg, "max_x"), (x, "max"))
T.link((y, "value"), (dg, "scroll_y"))
T.dlink((dg, "max_y"), (y, "max"))
T.link((dg, "row_size"), (rs, "value"))
T.link((dg, "column_size"), (cs, "value"))
T.dlink((dg, "hover_row"), (hr, "value"), "row {}".format)
T.dlink((dg, "hover_column"), (hc, "value"), "column {}".format)
colors = [ 
    [p, T.link((dg, t), (p, "value"))][0]
    for t in dg.traits().keys()
    for p in [AlphaColorPicker(description=t.replace('_color', '').replace('_', ' '))]
    if t.endswith('_color')
]
grid_playground = W.HBox([
    W.VBox([
        W.HBox([W.VBox([hr, hc]), x, cs, data.widget]), 
        dg, 
    ], layout=dict(flex="1")),
    W.VBox([y, rs, *colors]), 
])

In [ ]:
grid_playground